In [21]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import pandas as pd
import os
from backend.domain.model import Company, Filing

In [2]:
uow_instance = uow.UnitOfWork()
company = service.get_company_by_ticker('GES', uow_instance)


In [3]:
for filing in company.filings:
    print(filing.form)


10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K
10-Q
10-Q
10-Q
10-K


In [4]:
combined = service.get_consolidated_income_statements('GES', uow_instance, '10-Q')

Years covered in last filing (2024-12-06):
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Index(['2023-07-30:2023-10-28', '2024-08-04:2024-11-02'], dtype='object', name='date_range')
Error getting filing data for 0000912463 0001104659-10-033198 a10-10417_110q.htm: API error: 404 - {"status":404,"error":"The filer did not attach XBRL data to the filing. A conversion is not possible."}
Error getting filing data for 0000912463 0001104659-08-075833 a08-30083_110q.htm: API error: 404 - {"status":404,"error":"The filer did not attach XBRL data to the filing. A conversion is not possible."}
Error getting filing data for 0000912463 0001104659-08-058022 a08-23186_110q.htm: API error: 404 - {"status":404,"error":"The filer did not attach XBRL data to the filing. A conversion is not possible."}
Error getting filing data for 0000912463 0001104659-08-039393 a08-

In [5]:
combined.df

date_range,2022-01-30:2023-01-28,2023-01-29:2024-02-03,2024-02-04:2025-02-01,2020-02-02:2021-01-30,2021-01-31:2022-01-29,2018-02-04:2019-02-02,2019-02-03:2020-02-01,2016-01-31:2017-01-28,2017-01-29:2018-02-03,2014-02-02:2015-01-31,...,2011-07-31:2011-10-29,2012-07-29:2012-10-27,2011-05-01:2011-07-30,2012-04-29:2012-07-28,2011-01-30:2011-04-30,2012-01-29:2012-04-28,2009-08-02:2009-10-31,2010-08-01:2010-10-30,2009-05-03:2009-08-01,2010-05-02:2010-07-31
Impairment Charges,9.522000e+06,6.893500e+06,6.612000e+06,8.044200e+07,3.149000e+06,6.939000e+06,9.977000e+06,3.438500e+07,8.479000e+06,2.476600e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Goods Sold,1.538603e+09,1.553950e+09,1.694283e+09,1.179427e+09,1.422126e+09,1.670090e+09,1.662401e+09,1.445413e+09,1.534906e+09,1.549788e+09,...,NaN,NaN,3.797030e+08,3.838330e+08,NaN,NaN,NaN,NaN,NaN,NaN
Basic EPS,2.620000e+00,3.670000e+00,1.150000e+00,-1.270000e+00,2.650000e+00,1.700000e-01,1.350000e+00,2.700000e-01,-1.100000e-01,1.110000e+00,...,7.100000e-01,4.300000e-01,6.500000e-01,4.900000e-01,4.600000e-01,0.3,6.900000e-01,7.500000e-01,6.500000e-01,7.200000e-01
Diluted EPS,2.180000e+00,3.090000e+00,7.700000e-01,-1.270000e+00,2.570000e+00,1.600000e-01,1.330000e+00,2.700000e-01,-1.100000e-01,1.110000e+00,...,7.100000e-01,4.300000e-01,6.500000e-01,4.900000e-01,4.600000e-01,0.3,6.900000e-01,7.500000e-01,6.400000e-01,7.200000e-01
Lease Termination Gain/Loss,2.283500e+06,1.681000e+06,7.090000e+05,2.800500e+06,2.795000e+05,4.885000e+05,0.000000e+00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Debt Extinguishment Gain/Loss,0.000000e+00,-1.235100e+07,-1.952000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit,1.148747e+09,1.222580e+09,1.300990e+09,6.971020e+08,1.169505e+09,9.396040e+08,1.015708e+09,7.450400e+08,8.288480e+08,8.678850e+08,...,2.756010e+08,2.476090e+08,2.974560e+08,2.515600e+08,2.480300e+08,235076000.0,2.368910e+08,2.663970e+08,2.317770e+08,2.522360e+08
Operating Income,1.980460e+08,2.361350e+08,8.047300e+07,-8.707900e+07,2.537290e+08,4.670800e+07,1.237420e+08,5.361000e+07,7.027100e+07,1.430080e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investment Income,0.000000e+00,0.000000e+00,-4.090000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Income Tax Expense,3.650200e+07,2.541800e+07,9.695000e+06,-6.338000e+06,7.368000e+07,2.954200e+07,2.251300e+07,2.821200e+07,7.417200e+07,4.582400e+07,...,3.187700e+07,1.998900e+07,3.453400e+07,2.021200e+07,1.823700e+07,12806000.0,3.205400e+07,2.881800e+07,2.966200e+07,2.903000e+07


In [14]:
combined.df[sorted(combined.df.columns, key=lambda x: x.split(':')[0])].columns

Index(['2008-02-03:2009-01-31', '2009-02-01:2010-01-30',
       '2009-05-03:2009-08-01', '2009-08-02:2009-10-31',
       '2010-01-31:2011-01-29', '2010-05-02:2010-07-31',
       '2010-08-01:2010-10-30', '2011-01-30:2012-01-28',
       '2011-01-30:2011-04-30', '2011-05-01:2011-07-30',
       '2011-07-31:2011-10-29', '2012-01-29:2013-02-02',
       '2012-01-29:2012-04-28', '2012-04-29:2012-07-28',
       '2012-07-29:2012-10-27', '2013-02-03:2014-02-01',
       '2013-02-03:2013-05-04', '2013-05-05:2013-08-03',
       '2013-08-04:2013-11-02', '2014-02-02:2015-01-31',
       '2014-02-02:2014-05-03', '2014-05-04:2014-08-02',
       '2014-08-03:2014-11-01', '2015-02-01:2016-01-30',
       '2015-02-01:2015-05-02', '2015-05-03:2015-08-01',
       '2015-08-02:2015-10-31', '2016-01-31:2017-01-28',
       '2016-01-31:2016-04-30', '2016-05-01:2016-07-30',
       '2016-07-31:2016-10-29', '2017-01-29:2018-02-03',
       '2017-01-29:2017-04-29', '2017-04-30:2017-07-29',
       '2017-07-30:2017-10-28',

In [20]:
combined.df.empty

False

In [22]:
def check_coverage_continuity(combined) -> dict:
    if combined.df.empty:
        return {
            'has_gaps': False,
            'gaps': [],
            'annual_gaps': [],
            'quarterly_gaps': [],
            'coverage_summary': 'No data available'
        }
    
    from datetime import datetime, timedelta
    
    periods = []
    for col in combined.df.columns:
        if ':' not in col:
            continue
        
        try:
            start_str, end_str = col.split(':')
            start_date = pd.to_datetime(start_str)
            end_date = pd.to_datetime(end_str)
            duration_days = (end_date - start_date).days
            
            if 350 <= duration_days <= 380:
                period_type = 'annual'
            elif 85 <= duration_days <= 95:
                period_type = 'quarterly'
            else:
                period_type = 'other'
            
            periods.append({
                'start': start_date,
                'end': end_date,
                'duration_days': duration_days,
                'type': period_type,
                'label': col
            })
        except:
            continue
    
    if not periods:
        return {
            'has_gaps': False,
            'gaps': [],
            'annual_gaps': [],
            'quarterly_gaps': [],
            'coverage_summary': 'No valid date ranges found'
        }
    
    periods.sort(key=lambda x: x['start'])
    
    annual_periods = [p for p in periods if p['type'] == 'annual']
    quarterly_periods = [p for p in periods if p['type'] == 'quarterly']
    
    def find_gaps(period_list, max_gap_days):
        gaps = []
        for i in range(len(period_list) - 1):
            current_end = period_list[i]['end']
            next_start = period_list[i + 1]['start']
            gap_days = (next_start - current_end).days
            
            if gap_days > max_gap_days:
                gaps.append({
                    'gap_start': current_end + timedelta(days=1),
                    'gap_end': next_start - timedelta(days=1),
                    'gap_days': gap_days - 1,
                    'after_period': period_list[i]['label'],
                    'before_period': period_list[i + 1]['label']
                })
        
        return gaps
    
    annual_gaps = find_gaps(annual_periods, 1)
    quarterly_gaps = find_gaps(quarterly_periods, 1)
    
    all_gaps = []
    for i in range(len(periods) - 1):
        current_end = periods[i]['end']
        next_start = periods[i + 1]['start']
        gap_days = (next_start - current_end).days
        
        if gap_days > 1:
            all_gaps.append({
                'gap_start': current_end + timedelta(days=1),
                'gap_end': next_start - timedelta(days=1),
                'gap_days': gap_days - 1,
                'after_period': periods[i]['label'],
                'before_period': periods[i + 1]['label'],
                'after_type': periods[i]['type'],
                'before_type': periods[i + 1]['type']
            })
    
    coverage_start = periods[0]['start'].strftime('%Y-%m-%d')
    coverage_end = periods[-1]['end'].strftime('%Y-%m-%d')
    
    coverage_summary = f"Coverage from {coverage_start} to {coverage_end}"
    if annual_periods:
        coverage_summary += f"\n  - {len(annual_periods)} annual periods"
    if quarterly_periods:
        coverage_summary += f"\n  - {len(quarterly_periods)} quarterly periods"
    
    return {
        'has_gaps': len(all_gaps) > 0,
        'gaps': all_gaps,
        'annual_gaps': annual_gaps,
        'quarterly_gaps': quarterly_gaps,
        'coverage_summary': coverage_summary,
        'period_count': {
            'annual': len(annual_periods),
            'quarterly': len(quarterly_periods),
            'total': len(periods)
        },
        'date_range': {
            'start': coverage_start,
            'end': coverage_end
        }
    }


In [25]:
def create_implied_missing_quarters(combined) -> None:
        if combined.df.empty:
            return
        
        annual_columns = []
        quarterly_columns = []
        
        for col in combined.df.columns:
            start_str, end_str = col.split(':')
            start_date = pd.to_datetime(start_str)
            end_date = pd.to_datetime(end_str)
            period_days = (end_date - start_date).days
            
            if 350 < period_days < 380:
                annual_columns.append(col)
            elif 85 < period_days < 95:
                quarterly_columns.append(col)
        
        for annual_col in annual_columns:
            annual_start_str, annual_end_str = annual_col.split(':')
            annual_start = pd.to_datetime(annual_start_str)
            annual_end = pd.to_datetime(annual_end_str)
            
            quarters_in_annual = []
            for q_col in quarterly_columns:
                q_start_str, q_end_str = q_col.split(':')
                q_start = pd.to_datetime(q_start_str)
                q_end = pd.to_datetime(q_end_str)
                
                if q_start >= annual_start and q_end <= annual_end:
                    quarters_in_annual.append(q_col)
            
            if len(quarters_in_annual) == 3:
                quarter_dates = []
                for q_col in quarters_in_annual:
                    q_start_str, q_end_str = q_col.split(':')
                    quarter_dates.append((pd.to_datetime(q_start_str), pd.to_datetime(q_end_str)))
                
                quarter_dates.sort(key=lambda x: x[0])
                
                if quarter_dates[0][0] == annual_start:
                    implied_start = quarter_dates[-1][1] + pd.Timedelta(days=1)
                    implied_end = annual_end
                elif quarter_dates[-1][1] == annual_end:
                    implied_start = annual_start
                    implied_end = quarter_dates[0][0] - pd.Timedelta(days=1)
                else:
                    for i in range(len(quarter_dates) - 1):
                        if (quarter_dates[i+1][0] - quarter_dates[i][1]).days > 1:
                            implied_start = quarter_dates[i][1] + pd.Timedelta(days=1)
                            implied_end = quarter_dates[i+1][0] - pd.Timedelta(days=1)
                            break
                    else:
                        continue
                
                implied_col_name = f"{implied_start.strftime('%Y-%m-%d')}:{implied_end.strftime('%Y-%m-%d')}"
                
                if implied_col_name not in combined.df.columns:
                    combined.df[implied_col_name] = combined.df[annual_col].fillna(0)
                    for q_col in quarters_in_annual:
                        combined.df[implied_col_name] -= combined.df[q_col].fillna(0)

In [26]:
res = create_implied_missing_quarters(combined)

In [27]:
res

In [31]:
combined.df[sorted(combined.df.columns, key=lambda x: x.split(':')[0])]

date_range,2008-02-03:2009-01-31,2009-02-01:2010-01-30,2009-05-03:2009-08-01,2009-08-02:2009-10-31,2010-01-31:2011-01-29,2010-05-02:2010-07-31,2010-08-01:2010-10-30,2011-01-30:2012-01-28,2011-01-30:2011-04-30,2011-05-01:2011-07-30,...,2023-01-29:2024-02-03,2023-01-29:2023-04-29,2023-04-30:2023-07-29,2023-07-30:2023-10-28,2023-10-29:2024-02-03,2024-02-04:2025-02-01,2024-02-04:2024-05-04,2024-05-05:2024-08-03,2024-08-04:2024-11-02,2024-11-03:2025-02-01
Impairment Charges,2.444300e+07,4.708000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.893500e+06,1.917000e+06,2.611000e+06,1.718500e+06,6.470000e+05,6.612000e+06,1.120500e+06,2.288500e+06,1.095500e+06,2.107500e+06
Cost of Goods Sold,1.170762e+09,1.186979e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.797030e+08,...,1.553950e+09,3.378130e+08,3.700690e+08,3.600000e+08,4.860680e+08,1.694283e+09,3.438420e+08,4.126170e+08,4.166410e+08,5.211830e+08
Basic EPS,2.270000e+00,2.630000e+00,6.500000e-01,6.900000e-01,3.140000e+00,7.200000e-01,7.500000e-01,2.880000e+00,4.600000e-01,6.500000e-01,...,3.670000e+00,-2.200000e-01,7.300000e-01,1.040000e+00,2.120000e+00,1.150000e+00,2.400000e-01,-2.100000e-01,-4.600000e-01,1.580000e+00
Diluted EPS,2.250000e+00,2.610000e+00,6.400000e-01,6.900000e-01,3.110000e+00,7.200000e-01,7.500000e-01,2.860000e+00,4.600000e-01,6.500000e-01,...,3.090000e+00,-2.200000e-01,5.900000e-01,8.200000e-01,1.900000e+00,7.700000e-01,2.300000e-01,-2.800000e-01,-4.700000e-01,1.290000e+00
Lease Termination Gain/Loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.681000e+06,NaN,2.431000e+06,-5.370000e+05,-2.130000e+05,7.090000e+05,NaN,0.000000e+00,7.180000e+05,-9.000000e+03
Debt Extinguishment Gain/Loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.235100e+07,-7.696000e+06,0.000000e+00,0.000000e+00,-4.655000e+06,-1.952000e+06,-1.952000e+06,0.000000e+00,0.000000e+00,0.000000e+00
Gross Profit,9.226280e+08,9.414870e+08,2.317770e+08,2.368910e+08,1.082328e+09,2.522360e+08,2.663970e+08,1.156225e+09,2.480300e+08,2.974560e+08,...,1.222580e+09,2.319850e+08,2.944430e+08,2.911700e+08,4.049820e+08,1.300990e+09,2.481010e+08,3.199430e+08,3.218770e+08,4.110690e+08
Operating Income,NaN,NaN,NaN,NaN,4.213770e+08,NaN,NaN,3.993410e+08,NaN,NaN,...,2.361350e+08,-1.295200e+07,5.712700e+07,4.102700e+07,1.509330e+08,8.047300e+07,1.120100e+07,3.153000e+06,-9.068000e+06,7.518700e+07
Equity Investment Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,-4.090000e+05,NaN,-7.200000e+05,1.610000e+05,1.500000e+05
Income Tax Expense,1.037840e+08,1.155990e+08,2.966200e+07,3.205400e+07,1.268740e+08,2.903000e+07,2.881800e+07,1.286910e+08,1.823700e+07,3.453400e+07,...,2.541800e+07,-2.258000e+06,1.516500e+07,-1.827700e+07,3.078800e+07,9.695000e+06,-4.705000e+06,1.178900e+07,1.168700e+07,-9.076000e+06


In [8]:
combined.analyze_period_coverage()

{'coverage_by_year': {'2025': ['FY', 'Q1', 'Q2', 'Q3'],
  '2023': ['FY', 'Q1', 'Q2', 'Q3'],
  '2021': ['FY', 'Q1', 'Q2', 'Q3'],
  '2019': ['FY', 'Q1', 'Q2', 'Q3'],
  '2017': ['FY', 'Q1', 'Q2', 'Q3'],
  '2015': ['FY', 'Q1', 'Q2', 'Q3'],
  '2013': ['FY', 'Q1', 'Q2', 'Q3'],
  '2011': ['FY', 'Q2', 'Q3']},
 'missing_periods': {'2025': ['Q4'],
  '2023': ['Q4'],
  '2021': ['Q4'],
  '2019': ['Q4'],
  '2017': ['Q4'],
  '2015': ['Q4'],
  '2013': ['Q4'],
  '2011': ['Q1', 'Q4']},
 'has_complete_quarterly_coverage': {'2025': False,
  '2023': False,
  '2021': False,
  '2019': False,
  '2017': False,
  '2015': False,
  '2013': False,
  '2011': False},
 'years_analyzed': ['2011',
  '2013',
  '2015',
  '2017',
  '2019',
  '2021',
  '2023',
  '2025']}

In [9]:
for filing in combined.source_filings:
    try: 
        print(filing.cover_page.document_fiscal_period_focus, filing.cover_page.document_fiscal_year_focus)
    except:
        print(filing.form)

FY 2025
FY 2023
FY 2021
FY 2019
FY 2017
FY 2015
FY 2013
FY 2011
Q3 2025
Q2 2025
Q1 2025
Q3 2023
Q2 2023
Q1 2023
Q3 2021
Q2 2021
Q1 2021
Q3 2019
Q2 2019
Q1 2019
Q3 2017
Q2 2017
Q1 2017
Q3 2015
Q2 2015
Q1 2015
Q3 2013
Q2 2013
Q1 2013
Q3 2011
Q2 2011


In [10]:
sorted_columns = sorted(combined.df.columns, key=lambda x: x.split(':')[0])
print(sorted_columns)

['2008-02-03:2009-01-31', '2009-02-01:2010-01-30', '2009-05-03:2009-08-01', '2009-08-02:2009-10-31', '2010-01-31:2011-01-29', '2010-05-02:2010-07-31', '2010-08-01:2010-10-30', '2011-01-30:2012-01-28', '2011-01-30:2011-04-30', '2011-05-01:2011-07-30', '2011-07-31:2011-10-29', '2012-01-29:2013-02-02', '2012-01-29:2012-04-28', '2012-04-29:2012-07-28', '2012-07-29:2012-10-27', '2013-02-03:2014-02-01', '2013-02-03:2013-05-04', '2013-05-05:2013-08-03', '2013-08-04:2013-11-02', '2014-02-02:2015-01-31', '2014-02-02:2014-05-03', '2014-05-04:2014-08-02', '2014-08-03:2014-11-01', '2015-02-01:2016-01-30', '2015-02-01:2015-05-02', '2015-05-03:2015-08-01', '2015-08-02:2015-10-31', '2016-01-31:2017-01-28', '2016-01-31:2016-04-30', '2016-05-01:2016-07-30', '2016-07-31:2016-10-29', '2017-01-29:2018-02-03', '2017-01-29:2017-04-29', '2017-04-30:2017-07-29', '2017-07-30:2017-10-28', '2018-02-04:2019-02-02', '2018-02-04:2018-05-05', '2018-05-06:2018-08-04', '2018-08-05:2018-11-03', '2019-02-03:2020-02-01',

In [11]:
# Get data for first 4 filings
for filing in company.filings[:1]:
    filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    filing.filing_url = uow_instance.sec_filings.get_filing_url(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')


Loaded data for filing date 2025-04-11, form 10-K


In [12]:
res = company.select_filings_with_processing_pattern(company.filings,'10-Q')

AttributeError: 'NoneType' object has no attribute 'table'

In [31]:
for filing in res:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')

Loaded data for filing date 2024-12-06, form 10-Q
Loaded data for filing date 2024-09-06, form 10-Q
Loaded data for filing date 2024-06-07, form 10-Q
Loaded data for filing date 2022-12-02, form 10-Q
Loaded data for filing date 2022-09-02, form 10-Q
Loaded data for filing date 2022-06-02, form 10-Q
Loaded data for filing date 2020-12-04, form 10-Q
Loaded data for filing date 2020-09-04, form 10-Q
Loaded data for filing date 2020-06-11, form 10-Q
Loaded data for filing date 2018-12-07, form 10-Q
Loaded data for filing date 2018-09-06, form 10-Q
Loaded data for filing date 2018-06-07, form 10-Q
Loaded data for filing date 2016-12-06, form 10-Q
Loaded data for filing date 2016-09-01, form 10-Q
Loaded data for filing date 2016-06-02, form 10-Q
Loaded data for filing date 2014-12-09, form 10-Q
Loaded data for filing date 2014-09-05, form 10-Q
Loaded data for filing date 2014-06-06, form 10-Q
Loaded data for filing date 2012-12-05, form 10-Q
Loaded data for filing date 2012-09-05, form 10-Q
